In [2]:
import numpy as np
import pandas as pd

BESS_CAPACITY = 50  # MWh
MAX_C_RATE = 0.5    # 0.5C (Max charge/discharge rate is 25 MW)
RT_AFRR_PRICE = 250 # €/MWh for aFRR service delivery (High value RT)
DA_ARBITRAGE_THRESHOLD = 0.1 # Arbitrage requires 10% deviation from avg price

In [3]:
# --- 1. Scenario Generation ---

def generate_stochastic_scenarios(base_prices, num_scenarios=10, vol_factor=0.15):
    """
    Generates multiple probabilistic price paths (scenarios) for a 24-hour period.
    Each scenario has a Day-Ahead (DA) price and a realized Real-Time (RT) price.
    
    Args:
        base_prices (np.array): The base DA price forecast (e.g., from an ARIMA model).
        num_scenarios (int): Number of price paths to generate.
        vol_factor (float): Standard deviation factor for price noise (uncertainty).
        
    Returns:
        pd.DataFrame: A DataFrame with columns ['DA_Price_sX', 'RT_Price_sX'] for each scenario.
    """
    scenarios = pd.DataFrame(index=np.arange(len(base_prices)))
    
    for s in range(num_scenarios):
        # DA Price: Base forecast with a small random noise (market noise)
        da_noise = np.random.normal(0, base_prices.mean() * vol_factor * 0.2, base_prices.shape)
        da_price_s = np.maximum(5, base_prices + da_noise) # Ensure non-negative
        scenarios[f'DA_Price_s{s}'] = da_price_s
        
        # RT Price: Higher volatility and larger deviations from DA price
        rt_noise = np.random.normal(0, base_prices.mean() * vol_factor, base_prices.shape)
        rt_price_s = np.maximum(5, da_price_s + rt_noise)
        scenarios[f'RT_Price_s{s}'] = rt_price_s
        
    # Assign equal probability to each scenario
    scenarios['Probability'] = 1 / num_scenarios
    
    return scenarios.set_index(pd.to_datetime(np.arange(len(base_prices)), unit='h'))

In [4]:
# --- 2. Simplified Two-Stage Stochastic Solver ---

def solve_scenario_optimization(scenarios, capacity_reserved_rt):
    """
    Solves the BESS dispatch for the remaining capacity within a single scenario (Stage 2).
    
    The decision is: Reserve capacity_reserved_rt for RT (aFRR), use the rest for DA arbitrage.
    
    Args:
        scenarios (pd.DataFrame): DataFrame containing 'DA_Price_sX' and 'RT_Price_sX' columns.
        capacity_reserved_rt (float): MWh reserved for aFRR service.
        
    Returns:
        pd.DataFrame: Scenario-wise results including profits and capacity utilization.
    """
    HOURS = scenarios.shape[0]
    
    # Capacity available for DA Arbitrage (MWh)
    capacity_arb = BESS_CAPACITY - capacity_reserved_rt
    
    # Max hourly power rate for Arbitrage (MW)
    P_arb_max = MAX_C_RATE * capacity_arb 
    
    scenario_profits = []
    
    for col_prefix in [col.split('_s')[0] for col in scenarios.columns if 'DA_Price' in col]:
        
        DA_prices = scenarios[f'{col_prefix}_s{s}'].values
        RT_prices = scenarios[f'{col_prefix}_s{s}'].values # Using RT price for actual dispatch value

        avg_DA_price = DA_prices.mean()
        
        # Initial SOC assumed at 50% for arbitrage capacity
        soc_arb = capacity_arb * 0.5
        total_arb_profit = 0
        
        # --- Stage 2: DA Arbitrage Dispatch ---
        for hour in range(HOURS):
            price = DA_prices[hour]
            
            # Simplified Decision: Arbitrage is based on the DA forecast
            if price < avg_DA_price * (1 - DA_ARBITRAGE_THRESHOLD) and soc_arb < capacity_arb:
                # Charge (Buy Low)
                charge_power = min(capacity_arb - soc_arb, P_arb_max)
                total_arb_profit -= charge_power * price 
                soc_arb += charge_power * 0.95 # Simple efficiency
                
            elif price > avg_DA_price * (1 + DA_ARBITRAGE_THRESHOLD) and soc_arb > 0:
                # Discharge (Sell High)
                discharge_power = min(soc_arb, P_arb_max)
                total_arb_profit += discharge_power * 0.95 * price 
                soc_arb -= discharge_power
        
        # --- Stage 2: RT (aFRR) Revenue ---
        # The capacity reserved for aFRR is paid a fixed rate (RT_AFRR_PRICE)
        # We assume the RT capacity is successfully dispatched across the 24h
        
        # Total aFRR Revenue: Capacity reserved * Price/MWh/h * 24 hours
        total_rt_profit = capacity_reserved_rt * RT_AFRR_PRICE * HOURS
        
        scenario_profits.append({
            'Scenario': int(col_prefix.split('s')[-1]),
            'DA_Profit': total_arb_profit,
            'RT_Profit': total_rt_profit,
            'Total_Profit': total_arb_profit + total_rt_profit
        })

    return pd.DataFrame(scenario_profits)

In [ ]:
# Helper function for the Main script to run the Stage 1 decision
def evaluate_expected_profit(scenarios, capacity_reserved_rt):
    """Calculates the expected profit (Stage 1 objective) across all scenarios."""
    
    # Solve for all scenarios
    scenario_results = solve_scenario_optimization(scenarios, capacity_reserved_rt)
    
    # Calculate Expected Profit: Sum(Probability * Total_Profit_s)
    
    # The scenarios DataFrame has the probability column, but scenario_results doesn't.
    # We use the fixed scenario probability (1/num_scenarios)
    num_scenarios = len(scenario_results)
    probability = 1 / num_scenarios
    
    expected_profit = (scenario_results['Total_Profit'] * probability).sum()
    
    return expected_profit, scenario_results

